In [1]:
# -*- coding: utf-8 -*-

""" Deep Residual Network.
Applying a Deep Residual Network to CIFAR-10 Dataset classification task.
References:
    - K. He, X. Zhang, S. Ren, and J. Sun. Deep Residual Learning for Image
      Recognition, 2015.
    - Learning Multiple Layers of Features from Tiny Images, A. Krizhevsky, 2009.
Links:
    - [Deep Residual Network](http://arxiv.org/pdf/1512.03385.pdf)
    - [CIFAR-10 Dataset](https://www.cs.toronto.edu/~kriz/cifar.html)
"""

from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.activations import softmax, relu
from tflearn.layers.core import input_data, fully_connected
from tflearn.layers.conv import conv_2d, residual_block, global_avg_pool
from tflearn.layers.normalization import batch_normalization
from tflearn.optimizers import Momentum
from tflearn.layers.estimator import regression

# Residual blocks
# 32 layers: n=5, 56 layers: n=9, 110 layers: n=18
n = 2

# Choose binarization type
binarize = None

# Set model name
name='ResNet-2-basic'

# Data loading
from tflearn.datasets import cifar10
(X, Y), (testX, testY) = cifar10.load_data()
Y = tflearn.data_utils.to_categorical(Y, 10)
testY = tflearn.data_utils.to_categorical(testY, 10)

# Real-time data preprocessing
img_prep = tflearn.ImagePreprocessing()
img_prep.add_featurewise_zero_center(per_channel=True)

# Real-time data augmentation
img_aug = tflearn.ImageAugmentation()
img_aug.add_random_flip_leftright()

# % Residual Network
network = input_data(shape=[None, 32, 32, 3],
                     data_preprocessing=img_prep,
                     data_augmentation=img_aug)
network = conv_2d(network, 16, 3, regularizer='L2', activation='relu', weight_decay=0.0001, binarize=binarize)
network = residual_block(network, n, 16, binarize=binarize)
network = residual_block(network, 1, 32, downsample=True, binarize=binarize)
network = residual_block(network, n-1, 32, binarize=binarize)
network = residual_block(network, 1, 64, downsample=True, binarize=binarize)
network = residual_block(network, n-1, 64, binarize=binarize)
network = batch_normalization(network)
network = relu(network)
network = global_avg_pool(network)

# Regression
network = fully_connected(network, 10, activation=None, binarize=binarize)
network = softmax(network)
mom = tflearn.Momentum(0.1, lr_decay=0.1, decay_step=32000, staircase=True)
network = regression(network, optimizer=mom, loss='categorical_crossentropy')
# Training
model = tflearn.DNN(network, tensorboard_verbose=0)

model.fit(X, Y, n_epoch=15, validation_set=(testX, testY), show_metric=True, 
          batch_size=100, shuffle=True, run_id=name)

Training Step: 7499  | total loss: 0.54970 | time: 358.976s
| Momentum | epoch: 015 | loss: 0.54970 - acc: 0.8452 -- iter: 49900/50000
Training Step: 7500  | total loss: 0.52815 | time: 378.278s
| Momentum | epoch: 015 | loss: 0.52815 - acc: 0.8517 | val_loss: 0.65203 - val_acc: 0.7782 -- iter: 50000/50000
--


In [2]:
model.fit(X, Y, n_epoch=4, validation_set=(testX, testY), show_metric=True, 
          batch_size=100, shuffle=True, run_id=name)

Training Step: 7078  | total loss: 0.67836 | time: 382.781s
| Momentum | epoch: 015 | loss: 0.67836 - acc: 0.7757 -- iter: 49900/50000
Training Step: 7079  | total loss: 0.66133 | time: 405.392s
| Momentum | epoch: 015 | loss: 0.66133 - acc: 0.7831 | val_loss: 1.07522 - val_acc: 0.6431 -- iter: 50000/50000
--
